In [1]:
import numpy as np
import pandas as pd
#import matplotlib.pyplot as plt
#import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

#from imblearn.over_sampling import SMOTE
from sklearn.impute import KNNImputer
from sklearn.preprocessing import StandardScaler


import statsmodels.api as sm
import statsmodels.api as sm
from statsmodels.formula.api import logit

# 1. Import Data

## 1.1 Control Variables

In [4]:
def ConvertConVarTab(df, value_col_name):
  df = pd.melt(df, id_vars = ['Country Name'], value_vars = [1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004,  2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017], var_name = 'year', value_name = value_col_name)
  return df

ConVarTabs = ['GDPGR', 'GDPPC' , 'IFR' , 'IR', 'XM' , 'M2R', 'CPGDP', 'CG', 'CPG', 'CPI']

ConVar = pd.DataFrame()

for i in range(len(ConVarTabs)):
  df = pd.read_excel('Control Variables_new.xlsx', ConVarTabs[i])
  df = ConvertConVarTab(df, ConVarTabs[i])
  if i == 0:
    ConVar = df
  else:
    ConVar = pd.concat([ConVar, df.iloc[:,2]], axis=1)


In [5]:
ConVar['Country Name'].nunique()

31

## 1.2 Capital Controls Variables

In [7]:
CapConVar = pd.read_excel('F_CC.xlsx', 'kai and kao(self)')

In [8]:
CapConVar = CapConVar.iloc[:, 2:]

In [9]:
CapConVar

,eqi,eqo,dbi,dbo,fii,fio,kai,kao
0,0.0,1.0,0.0,2.0,0,0,0.000000,0.333333
1,0.0,1.0,0.0,0.0,0,0,0.777778,0.666667
2,0.0,1.0,0.0,1.0,0,0,0.500000,0.583333
3,0.0,1.0,0.0,1.5,1,0,1.000000,0.833333
4,0.0,1.0,0.0,1.5,1,0,1.000000,1.000000
...,...,...,...,...,...,...,...,...
708,0.0,0.0,0.0,0.0,0,0,0.700000,0.750000
709,0.0,0.0,0.0,0.0,0,0,0.650000,0.750000
710,0.0,0.0,0.0,0.0,0,0,0.700000,0.950000
711,0.0,0.0,0.0,0.0,0,0,0.850000,0.500000


## 1.3 Institutional Quality Variables

In [11]:
InsQuaVar = pd.read_excel('Institutional Quality Variables.xlsx', 'IQ')

In [12]:
InsQuaVar = pd.melt(InsQuaVar, id_vars = ['Country Name'], value_vars = [1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004,  2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017], var_name = 'year', value_name ='INS')

In [13]:
InsQuaVar

,Country Name,year,INS
0,Argentina,1995,2.000000
1,Brazil,1995,3.000000
2,Bulgaria,1995,2.000000
3,Chile,1995,2.000000
4,China,1995,2.666667
...,...,...,...
708,Thailand,2017,2.000000
709,Turkiye,2017,2.000000
710,Ukraine,2017,1.000000
711,"Venezuela, RB",2017,1.000000


## 1.4 Macroprudential Policy

In [15]:
Macro_Policy_NFX = pd.read_excel('MaPs data.xlsx', 'NFXs')
Macro_Policy_FX = pd.read_excel('MaPs data.xlsx', 'FXs')
Macro_Policy = pd.read_excel('MaPs data.xlsx', 'MaPs')

In [16]:
Macro_Policy

,Country,IND,1995,1996,1997,1998,1999,2000,2001,2002,...,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017
0,Argentina,MaPs,NaN,NaN,NaN,NaN,NaN,3.0,4.0,4.0,...,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0
1,Brazil,MaPs,NaN,NaN,NaN,NaN,NaN,4.0,4.0,4.0,...,4.0,4.0,4.0,4.0,4.0,5.0,5.0,5.0,5.0,5.0
2,Bulgaria,MaPs,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0,...,3.0,3.0,3.0,3.0,3.0,3.0,3.0,2.0,2.0,3.0
3,Chile,MaPs,NaN,NaN,NaN,NaN,NaN,6.0,6.0,6.0,...,6.0,6.0,7.0,7.0,7.0,7.0,7.0,7.0,8.0,7.0
4,China,MaPs,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0,...,4.0,4.0,4.0,4.0,5.0,7.0,8.0,10.0,10.0,10.0
5,Colombia,MaPs,NaN,NaN,NaN,NaN,NaN,6.0,6.0,6.0,...,7.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0
6,Czech Republic,MaPs,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,3.0,3.0,4.0,5.0
7,Egypt,MaPs,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,2.0
8,Estonia,MaPs,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,3.0,4.0,4.0
9,Hungary,MaPs,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,...,0.0,0.0,3.0,4.0,4.0,4.0,4.0,4.0,4.0,5.0


In [17]:
Macro_Policy.drop('IND', axis = 1, inplace = True)
Macro_Policy = pd.melt(Macro_Policy, id_vars = ['Country'], value_vars = [1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004,  2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017], var_name = 'year', value_name ='MAR')

In [18]:
Macro_Policy_NFX.drop('IND', axis = 1, inplace = True)
Macro_Policy_NFX = pd.melt(Macro_Policy_NFX, id_vars = ['Country'], value_vars = [1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004,  2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017], var_name = 'year', value_name ='NFX')

In [19]:
Macro_Policy_FX.drop('IND', axis = 1, inplace = True)
Macro_Policy_FX = pd.melt(Macro_Policy_FX, id_vars = ['Country'], value_vars = [1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004,  2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017], var_name = 'year', value_name ='FX')

In [20]:
Macro_Policy.shape

(713, 3)

## 1.5 Exchange Rate Regime 

In [22]:
Exg_Rate = pd.read_excel('Exchange rate_new.xlsx', 'Exchange_Rate_Regime')

In [23]:
Exg_Rate.head()

,Country,year,ER1,ER2,ER3,ER4
0,Argentina,1995,1,0,0,0
1,Brazil,1995,0,0,1,0
2,Bulgaria,1995,0,0,1,0
3,Chile,1995,0,0,1,0
4,China,1995,0,1,0,0


In [24]:
Exg_Rate.shape

(713, 6)

## 1.6 Banking Crises Variables (Dependent Variables) And Generate The Completed Dataframe


In [26]:
BanCriVar = pd.read_excel('L Banking Crises New.xlsx', 'BC 4 categories')

In [27]:
BanCriVar

,country,year,LBC,LBC_2,LBC_3,LBC_4,RBC
0,Argentina,1995,NaN,1.0,NaN,NaN,NaN
1,Brazil,1995,NaN,0.0,NaN,NaN,NaN
2,Bulgaria,1995,NaN,0.0,NaN,NaN,NaN
3,Chile,1995,NaN,0.0,NaN,NaN,NaN
4,China,1995,NaN,0.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...
708,Thailand,2017,0.0,0.0,0.0,0.0,NaN
709,Turkey,2017,0.0,0.0,0.0,0.0,NaN
710,Ukraine,2017,0.0,0.0,0.0,0.0,NaN
711,Venezuela,2017,0.0,0.0,0.0,0.0,NaN


In [28]:
banking_crises_data = pd.concat([BanCriVar, CapConVar, InsQuaVar.drop(columns = ['Country Name', 'year']), ConVar.drop(columns = ['Country Name', 'year']), Macro_Policy.drop(columns = ['Country', 'year']), Macro_Policy_FX.drop(columns = ['Country', 'year']), Macro_Policy_NFX.drop(columns = ['Country', 'year']), Exg_Rate.drop(columns = ['Country', 'year'])], axis = 1)

In [29]:
banking_crises_data = banking_crises_data.sort_values(by = ['country', 'year'])

In [30]:
banking_crises_data.columns

Index(['country', 'year', 'LBC', 'LBC_2', 'LBC_3', 'LBC_4', 'RBC', 'eqi',
       'eqo', 'dbi', 'dbo', 'fii', 'fio', 'kai', 'kao', 'INS', 'GDPGR',
       'GDPPC', 'IFR', 'IR', 'XM', 'M2R', 'CPGDP', 'CG', 'CPG', 'CPI', 'MAR',
       'FX', 'NFX', 'ER1', 'ER2', 'ER3', 'ER4'],
      dtype='object')

In [31]:
pd.options.display.max_columns = None

In [32]:
banking_crises_data.tail()

,country,year,LBC,LBC_2,LBC_3,LBC_4,RBC,eqi,eqo,dbi,dbo,fii,fio,kai,kao,INS,GDPGR,GDPPC,IFR,IR,XM,M2R,CPGDP,CG,CPG,CPI,MAR,FX,NFX,ER1,ER2,ER3,ER4
588,Zimbabwe,2013,0.0,0.0,0.0,0.0,0.0,2.0,2.0,6.0,7.0,1,1,0.0,0.0,1.5,1.989493,1435.062663,0.016350,1.525435,-13.874382,8.185757,18.713554,-1.408708,-1.368733,109.075219,NaN,NaN,NaN,1,0,0,0
619,Zimbabwe,2014,0.0,0.0,0.0,0.0,0.0,1.0,2.0,5.0,5.5,1,0,0.0,0.0,1.5,2.376929,1443.617777,-0.001978,8.787605,-11.971153,12.030612,19.187374,0.478459,0.473820,108.859485,NaN,NaN,NaN,1,0,0,0
650,Zimbabwe,2015,0.0,0.0,0.0,0.0,NaN,1.5,2.0,3.0,7.0,1,1,0.0,0.0,1.5,1.779873,1445.069702,-0.024310,8.144428,-7.998095,10.898046,18.044351,-0.894686,-1.143022,106.213145,NaN,NaN,NaN,1,0,0,0
681,Zimbabwe,2016,0.0,0.0,0.0,0.0,NaN,1.5,1.0,5.5,2.5,1,0,0.0,0.0,1.5,0.755869,1433.608776,-0.015437,4.998563,-3.394138,13.846696,17.018645,-1.217123,-1.025707,104.573565,NaN,NaN,NaN,0,0,0,1
712,Zimbabwe,2017,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0,0,0.0,0.0,1.5,4.709492,1479.376056,0.008940,3.742846,-1.542184,21.313759,16.758158,-0.223454,NaN,105.508414,NaN,NaN,NaN,0,0,0,1


# 2. Preprocessing the Data

## 2.1 Drop Columns

In [141]:
## Drop Dependent Variables other than LBC_2

df_prep = banking_crises_data.copy()

df_prep.drop(['LBC', 'LBC_3', 'LBC_4', 'RBC'], axis = 1, inplace = True)

In [143]:
## Drop the Zimbabwe

df_prep = df_prep[df_prep['country'] != 'Zimbabwe']

In [145]:
## Check the multilinearity of variables and drop those having perfect linearity

cor_matrix_X = df_prep.iloc[:, 2:].corr()

threshold = 0.8  # Set the threshold
high_corr = ((cor_matrix_X.abs() >= threshold) & (cor_matrix_X.abs() < 1.0))

# Filter and find features with high correlation
high_corr_pairs = [(i, j) for i in high_corr.columns for j in high_corr.index if high_corr.loc[i, j]]
print("Highly correlated pairs (above threshold of {}):".format(threshold))
for pair in high_corr_pairs:
    print(pair, "with correlation", cor_matrix_X.loc[pair[0], pair[1]])

Highly correlated pairs (above threshold of 0.8):
('eqi', 'dbi') with correlation 0.8426185004684841
('eqo', 'dbo') with correlation 0.8650628023654721
('dbi', 'eqi') with correlation 0.8426185004684841
('dbi', 'dbo') with correlation 0.88381759921724
('dbo', 'eqo') with correlation 0.8650628023654721
('dbo', 'dbi') with correlation 0.88381759921724
('kai', 'kao') with correlation 0.8738674185668076
('kao', 'kai') with correlation 0.8738674185668076
('MAR', 'NFX') with correlation 0.8412736931060817
('NFX', 'MAR') with correlation 0.8412736931060817


## 2.2 Dealing with Missing Values

In [148]:
df_prep.isnull().sum()

country      0
year         0
LBC_2       43
eqi          0
eqo          0
dbi          0
dbo          0
fii          0
fio          0
kai          0
kao          0
INS          0
GDPGR        0
GDPPC        0
IFR          0
IR          30
XM           0
M2R          5
CPGDP        3
CG          10
CPG         62
CPI          0
MAR        150
FX         150
NFX        150
ER1          0
ER2          0
ER3          0
ER4          0
dtype: int64

In [150]:
## Using forward and backward imputation for the variables with missing values

df_prep.set_index(['country', 'year'], inplace=True)

expVar_missvalues = ['IR', 'M2R', 'CPGDP', 'CG', 'CPG', 'MAR', 'FX', 'NFX']
for i in expVar_missvalues:
  df_prep[i] = df_prep.groupby('country')[i].ffill()
  df_prep[i] = df_prep.groupby('country')[i].bfill()

In [152]:
df_prep.isnull().sum()

LBC_2    43
eqi       0
eqo       0
dbi       0
dbo       0
fii       0
fio       0
kai       0
kao       0
INS       0
GDPGR     0
GDPPC     0
IFR       0
IR        0
XM        0
M2R       0
CPGDP     0
CG        0
CPG       0
CPI       0
MAR       0
FX        0
NFX       0
ER1       0
ER2       0
ER3       0
ER4       0
dtype: int64

## Create Lag for All Independent Variables

In [155]:
X = ['eqi', 'eqo', 'dbi', 'dbo', 'fii', 'fio', 'kai', 'kao', 'INS', 'MAR', 'FX', 'NFX', 'ER1', 'ER2', 'ER3', 'ER4', 'GDPGR',
       'GDPPC', 'IFR', 'IR', 'XM', 'M2R', 'CPGDP', 'CG', 'CPG', 'CPI']

for i in X:
  df_prep[f'lag_{i}'] = df_prep.groupby('country')[i].shift(1)
df_prep.dropna(inplace=True)

In [157]:
## Drop the original X

df_prep.drop(X, axis = 1, inplace = True)

In [159]:
df_prep.shape

(617, 27)

## Scale the X

In [162]:
## Using StandardScaler to Partial Normalize The X

df_prep.reset_index(inplace = True)

partial_X = df_prep[['lag_GDPGR', 'lag_GDPPC', 'lag_IR', 'lag_XM', 'lag_IFR', 'lag_M2R', 'lag_CPGDP', 'lag_CG', 'lag_CPG']]
y = df_prep['LBC_2']

scalar = StandardScaler()
X_scaled = scalar.fit_transform(partial_X)


df_prep_scaled = pd.concat([df_prep[['lag_eqi', 'lag_eqo', 'lag_dbi', 'lag_dbo', 'lag_fii', 'lag_fio',
       'lag_kai', 'lag_kao','lag_INS', 'lag_MAR', 'lag_FX', 'lag_NFX', 'lag_ER1', 'lag_ER2', 'lag_ER3', 'lag_ER4']], pd.DataFrame(X_scaled, columns = partial_X.columns), y], axis = 1)


In [164]:
df_prep_scaled['lag_kai:lag_INS'] = df_prep_scaled['lag_kai'] * df_prep_scaled['lag_INS']
df_prep_scaled['lag_kai:lag_MAR'] = df_prep_scaled['lag_kai'] * df_prep_scaled['lag_MAR']
df_prep_scaled['lag_kai:lag_ER3'] = df_prep_scaled['lag_kai'] * df_prep_scaled['lag_ER3']
df_prep_scaled['lag_kai:lag_ER4'] = df_prep_scaled['lag_kai'] * df_prep_scaled['lag_ER4']
df_prep_scaled['lag_kao:lag_INS'] = df_prep_scaled['lag_kao'] * df_prep_scaled['lag_INS']
df_prep['lag_kai:lag_INS'] = df_prep['lag_kai'] * df_prep['lag_INS']
df_prep['lag_kao:lag_INS'] = df_prep['lag_kao'] * df_prep['lag_INS']
df_prep['lag_kai:lag_MAR'] = df_prep['lag_kai'] * df_prep['lag_MAR']
df_prep['lag_kao:lag_MAR'] = df_prep['lag_kao'] * df_prep['lag_MAR']
df_prep['lag_kai:lag_ER4'] = df_prep['lag_kai'] * df_prep['lag_ER4']
df_prep['lag_kai:lag_ER3'] = df_prep['lag_kai'] * df_prep['lag_ER3']

In [166]:
df_prep_scaled.columns

Index(['lag_eqi', 'lag_eqo', 'lag_dbi', 'lag_dbo', 'lag_fii', 'lag_fio',
       'lag_kai', 'lag_kao', 'lag_INS', 'lag_MAR', 'lag_FX', 'lag_NFX',
       'lag_ER1', 'lag_ER2', 'lag_ER3', 'lag_ER4', 'lag_GDPGR', 'lag_GDPPC',
       'lag_IR', 'lag_XM', 'lag_IFR', 'lag_M2R', 'lag_CPGDP', 'lag_CG',
       'lag_CPG', 'LBC_2', 'lag_kai:lag_INS', 'lag_kai:lag_MAR',
       'lag_kai:lag_ER3', 'lag_kai:lag_ER4', 'lag_kao:lag_INS'],
      dtype='object')

In [168]:
df_prep['lag_GDPPC'] = df_prep['lag_GDPPC']/1000

In [250]:
X = df_prep[[ 'lag_kao', 'lag_kao:lag_INS', 'lag_INS', 'lag_MAR', 'lag_ER2', 'lag_ER3', 'lag_ER4',  'lag_GDPGR', 'lag_GDPPC', 'lag_IFR',
       'lag_IR', 'lag_XM', 'lag_M2R', 'lag_CPGDP']]
y = df_prep_scaled['LBC_2']

In [252]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(class_weight='balanced', solver='liblinear')  # liblinear supports small data
model.fit(X, y)

LogisticRegression(class_weight='balanced', solver='liblinear')

In [254]:
from scipy.stats import norm

# Get model coefficients and intercept
coefs = np.concatenate([model.intercept_, model.coef_.flatten()])

# Add intercept column to X
X_design = np.hstack((np.ones((X.shape[0], 1)), X.values))

# Predicted probabilities
preds = model.predict_proba(X)[:, 1]

# Diagonal weight matrix
V = np.diag(preds * (1 - preds))

# Hessian approximation: (X'VX)^-1
cov_matrix = np.linalg.inv(X_design.T @ V @ X_design)

# Standard errors
standard_errors = np.sqrt(np.diag(cov_matrix))

# Wald z-scores
z_scores = coefs / standard_errors

# Two-tailed p-values
p_values = 2 * (1 - norm.cdf(np.abs(z_scores)))

# Odds ratios
odds_ratios = np.exp(coefs)

# Variable names (add intercept manually)
var_names = ['Intercept'] + list(X.columns)

# Combine into DataFrame
results_df = pd.DataFrame({
    'Variable': var_names,
    'Coefficient': coefs,
    'Std. Error': standard_errors,
    'z': z_scores,
    'P>|z|': p_values,
    'Odds Ratio': odds_ratios
}).round(4)

In [256]:
results_df

,Variable,Coefficient,Std. Error,z,P>|z|,Odds Ratio
0,Intercept,-1.9378,0.6914,-2.8027,0.0051,0.1440
1,lag_kao,2.9947,0.8864,3.3785,0.0007,19.9785
2,lag_kao:lag_INS,-0.9336,0.3884,-2.4034,0.0162,0.3932
3,lag_INS,0.2889,0.2533,1.1403,0.2542,1.3349
4,lag_MAR,-0.1807,0.0632,-2.8597,0.0042,0.8347
5,lag_ER2,0.3337,0.4087,0.8164,0.4143,1.3961
6,lag_ER3,1.0934,0.4685,2.3337,0.0196,2.9843
7,lag_ER4,-1.4079,0.3787,-3.7176,0.0002,0.2447
8,lag_GDPGR,-0.1234,0.0312,-3.9515,0.0001,0.8839
9,lag_GDPPC,-0.0631,0.0238,-2.6462,0.0081,0.9389


In [238]:
probs = model.predict_proba(X)[:, 1]

In [240]:
coef = model.coef_.flatten()  # shape (n_features,)

# Choose the index of your capital controls variable
j = X.columns.get_loc('lag_INS')  # if X is a DataFrame

# Compute marginal effect for capital controls
marginal_effects = probs * (1 - probs) * coef[j]

# Compute average partial effect
APE = np.mean(marginal_effects)

print(f"Average Partial Effect of Capital Controls: {APE:.4f}")

Average Partial Effect of Capital Controls: -0.0435


In [214]:
df_prep['lag_INS'].mean()

2.2445299837925448

In [670]:
df_prep_temp = df_prep.copy()

df_prep_temp['lag_kao_change'] = df_prep.groupby('country')['lag_kao'].diff()

In [672]:
df_prep_temp = df_prep_temp.groupby('country')['lag_kao_change'].max()

In [696]:
df_prep['lag_MAR'].max()

10.0